In [2]:
# Run in python console
import nltk; nltk.download('stopwords')
import pandas as pd
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


#For stopwords
from nltk.corpus import stopwords

# spacy for lemmatization
import spacy

#For tokenizing
from nltk.tokenize import sent_tokenize

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#For Sentiment Analysis
from textblob import TextBlob


#For page rank
import networkx as nx
import operator
import itertools
import scipy as sp
from scipy.sparse import csc_matrix 
from nltk import bigrams
from igraph import *

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/srisuma/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [38]:
dataFrame=pd.read_csv('AspReviews.csv',error_bad_lines=False)
dataFrame=dataFrame.drop(columns='Unnamed: 0')
dataFrame

,Reviews
0,"nice phone, nice grade from pantach revue. Ver..."
1,already phone with problems know stated used ...
2,charging port loose that soldered. Then neede...
3,Phone looks good wouldnt stay charged battery....
4,originally using Samsung Galaxy Sprint wanted...
...,...
130086,This second junk Convoy have owned first less ...
130087,This cell phone provides just service require ...
130088,They said phone normal wear. Phone completely ...
130089,Item listed. When tried activate phone could a...


In [14]:
overall_rating=3.9

In [41]:
# NLTK Stop words

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu','phone','cellphone','smartphone','randomly','mobile','also'])


In [42]:
#
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True)) 
        
data_words = list(sent_to_words(dataFrame['Reviews']))
print(data_words[:1])


[['nice', 'phone', 'nice', 'grade', 'from', 'pantach', 'revue', 'very', 'clean', 'easy', 'never', 'android', 'phone', 'they', 'fantastic', 'least', 'perfect', 'size', 'surfing', 'social', 'media', 'great', 'phone', 'samsung']]


In [43]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['nice', 'phone', 'nice', 'grade', 'from', 'pantach', 'revue', 'very', 'clean', 'easy', 'never', 'android', 'phone', 'they', 'fantastic', 'least', 'perfect', 'size', 'surfing', 'social_media', 'great', 'phone', 'samsung']


In [44]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [46]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])
print(len(data_words))

[['nice', 'never', 'android', 'fantastic', 'least', 'perfect', 'size', 'surfing', 'social_media', 'great']]
130091


In [47]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])
id2word[0]

# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)]]


[[('android', 1),
  ('fantastic', 1),
  ('great', 1),
  ('least', 1),
  ('never', 1),
  ('nice', 1),
  ('perfect', 1),
  ('size', 1),
  ('social_media', 1),
  ('surfing', 1)]]

In [48]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10,
                                 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.087*"overall" + 0.075*"drop" + 0.046*"beautiful" + 0.040*"edge" + '
  '0.040*"spec" + 0.036*"real" + 0.029*"axon" + 0.027*"inch" + 0.022*"damage" '
  '+ 0.019*"otherwise"'),
 (1,
  '0.051*"good" + 0.045*"great" + 0.039*"camera" + 0.027*"use" + 0.026*"well" '
  '+ 0.024*"look" + 0.024*"really" + 0.020*"quality" + 0.019*"love" + '
  '0.019*"need"'),
 (2,
  '0.057*"awesome" + 0.048*"hope" + 0.046*"be" + 0.029*"instead" + '
  '0.026*"iphone" + 0.020*"image" + 0.020*"difficult" + 0.018*"true" + '
  '0.017*"rear" + 0.016*"stream"'),
 (3,
  '0.071*"work" + 0.060*"battery" + 0.043*"time" + 0.040*"take" + '
  '0.038*"charge" + 0.036*"come" + 0.025*"problem" + 0.024*"card" + 0.023*"go" '
  '+ 0.022*"can"'),
 (4,
  '0.035*"even" + 0.029*"buy" + 0.026*"issue" + 0.022*"update" + 0.019*"give" '
  '+ 0.019*"month" + 0.018*"receive" + 0.018*"year" + 0.017*"review" + '
  '0.016*"product"'),
 (5,
  '0.132*"screen" + 0.044*"sound" + 0.034*"button" + 0.032*"case" + '
  '0.032*"charger" + 0.031*"

In [49]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.43959081363473

Coherence Score:  0.34086974298953504


In [51]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.246100  0.049493       1        1  25.540453
3     -0.167474 -0.184095       2        1  19.795033
4     -0.193521  0.106914       3        1  17.832323
8     -0.170916 -0.110112       4        1  13.624933
5      0.053824  0.403422       5        1   8.767350
7      0.127831 -0.056553       6        1   4.069320
9      0.136656 -0.047778       7        1   3.556839
2      0.151977 -0.049780       8        1   2.430475
0      0.152030 -0.051969       9        1   2.348155
6      0.155692 -0.059541      10        1   2.035111, topic_info=                    Term          Freq         Total Category  logprob  \
103               screen  49301.000000  49301.000000  Default  30.0000   
32                  work  59904.000000  59904.000000  Default  29.0000   
400                 call  28447.000000  28447.000000  Default  28.0000   
33               battery  52119.000000  52119.000000  Default  27.0000   
17                  good  55138.000000  55138.000000  Default  26.0000   
...                  ...           ...           ...      ...      ...   
903              confirm    843.490051    844.240601  Topic10  -4.6309   
752               letter    817.173035    817.923523  Topic10  -4.6626   
1384                here    788.614990    789.365601  Topic10  -4.6981   
2328  fingerprint_sensor    718.232849    718.983398  Topic10  -4.7916   
373                sized    691.597534    692.348022  Topic10  -4.8294   

      loglift  
103   30.0000  
32    29.0000  
400   28.0000  
33    27.0000  
17    26.0000  
...       ...  
903    3.8937  
752    3.8937  
1384   3.8937  
2328   3.8936  
373    3.8935  

[354 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
136       3  0.999889       able
210      10  0.999696     access
3244      8  0.999007     active
1358      8  0.998770     actual
564       4  0.999839   actually
...     ...       ...        ...
333       2  0.006805       will
333       3  0.993156       will
1100      9  0.999546  wonderful
32        2  0.999988       work
334       3  0.999966       year

[358 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 5, 9, 6, 8, 10, 3, 1, 7])

In [ ]:
########## TESTING ########################

In [3]:

keys={'battery':['port','drain','battery','charge','charger','charging point','time'],'screen':['screen','display','button','glass','touch','keyboard'],'camera':['camera','quality','size','picture','pictures','photos']}
aspects=['battery','screen','camera']

In [4]:
testData=pd.read_csv('LabeledReviews.csv',error_bad_lines=False)
print(testData)
actualTopicLabels=testData['label2']
actualSentimentLabels=testData['label1']
#actualRatings=testData['label3']

                                               Reviews    label1   label2  \
0    Its not bad for the price, but there is lots o...   Neutral  Battery   
1    Iphone came in very good condition as describe...  Negative  Battery   
2    i bought i phone 4 and this i phone supposed t...  Negative   Camera   
3    Needed a replacement phone for my daughter Thi...  Positive   Screen   
4    It didnt work. The microphone wouldnt work hal...  Negative   Screen   
..                                                 ...       ...      ...   
313  I have one and my son had to have one. I have ...  Negative   Camera   
314  well Ive been using this iphone for 10days now...  Positive   Screen   
315  I bought this iPhone 2 years ago and Im still ...  Positive  Battery   
316  Went with an iphone 4 32gb when my ipod 5th ge...  Positive   Screen   
317  can you help me to unlock it because it does n...   Neutral   Camera   

     label3  
0         3  
1         1  
2         4  
3         5  
4    

In [5]:
data=testData.Reviews.values.tolist()
absa= dict()
unique_aspSent=[]
sentences_count=0
for aspect in aspects:
    for review in data:
        sentences=sent_tokenize(review)
        sentences_count+=len(sentences)
        for sentence in sentences:
                if any(word in sentence for word in keys[aspect]):
                    unique_aspSent.append(sentence)
                    if aspect in absa:
                        absa[aspect].append(sentence)                            
                    else:
                        absa[aspect] = [sentence]
                   
                        
                        
 

     
print("Aspect Specific sentences:",absa)
sentences_count=len(unique_aspSent) 
#print("Number of Sentences:",sentences_count)

Aspect Specific sentences: {'battery': ['Another thing, it says it comes with a charger and it didnt!', '!battery  is average', 'Iphone came in very good condition as described, but the charger was not working well.', 'Besides that, everything was ok.battery is worst.battery is draining very fast', 'The microphone wouldnt work half the time and the person couldnt hear me.', 'Got this phone because my android smart phone would only hold a charge for 5 hours.', 'On the plus side, this phone holds a charge for days, on the negative side, folks dont get my texts sometimes, or I dont get their texts sometimes, and it ALWAYS cuts my call off when Ive only been talking for a few minutes.', 'This one is garbage, sorry I bought it.battery is draining very fast and worst battery.', 'The purchase of this iphone 4 has caused me so much hassle and waste of time.', 'I had contacted mobile karma and they have no means to settling or solving this issue.it is taking lot of time for getting charged and 

In [300]:
absa_scores = dict()

for k, v in absa.items():
    s=""
    s=s.join(v)
    blob=TextBlob(s)
    temp=0
    for sent in blob.sentences:
        score = sent.sentiment.polarity
        temp+=score
    l=len(blob.sentences)
    absa_scores[k]=temp/l
print("Sentiment scores:")
print(absa_scores)
for key in absa_scores:
    sc=absa_scores[key]
    sc+=1
    sc*=2.5
    absa_scores[key]=sc
    
print("Normalized Aspect Rating")
print(absa_scores)

Sentiment scores:
{'battery': 0.15768105067130264, 'screen': 0.17434169589600304, 'camera': 0.30949247964825227}
Normalized Aspect Rating
{'battery': 2.894202626678257, 'screen': 2.935854239740008, 'camera': 3.273731199120631}


In [320]:
# topic labelling
predictedTopicLabels=[]
value=dict();
weight={"battery":0.178,"charge":0.079,"life":0.078,"charging port": 0.020,"drain":0.078,"port": 0.018,"charger":0.034,"camera":0.136,"picture":0.045,"pictures":0.045,"quality":0.068,"photos":0.098,"screen":0.155,"time":0.092,"touch":0.021,"button":0.031,"display":0.025,"glass":0.035,"keyboard":0.013};

topic=""
for review in data:
    value['battery']=0;value['screen']=0;value['camera']=0;
    maxi=0
    for k,v in keys.items():
        cnt = list(set(v).intersection(set(review.split())))
        for key in cnt:
            value[k]+=weight[key]      
    predictedTopicLabels.append(max(value,key = value.get))
   
        
for aspect in aspects:
    temp=absa_scores[aspect]
    
print(len(predictedTopicLabels),len(actualTopicLabels))

318 318


In [321]:
c=0
for i in range(318):
    print(actualTopicLabels[i].lower(),predictedTopicLabels[i],actualTopicLabels[i].lower()==predictedTopicLabels[i])
    actualTopicLabels[i]=actualTopicLabels[i].lower()
    if actualTopicLabels[i].lower()==predictedTopicLabels[i]:
        c+=1
print(c)

battery battery True
battery battery True
camera camera True
screen battery False
screen battery False
camera camera True
screen camera False
camera camera True
camera camera True
screen battery False
screen battery False
camera battery False
battery battery True
battery battery True
camera camera True
battery battery True
screen screen True
camera camera True
screen screen True
screen screen True
screen screen True
camera camera True
battery battery True
battery battery True
camera battery False
battery screen False
battery camera False
camera battery False
screen screen True
camera camera True
battery battery True
battery camera False
battery battery True
screen battery False
screen camera False
screen battery False
screen screen True
battery battery True
camera camera True
battery battery True
camera camera True
camera camera True
camera camera True
battery battery True
camera battery False
battery battery True
screen screen True
camera camera True
camera camera True
screen screen T

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [322]:
#confusion matrix
from sklearn import metrics
print(metrics.confusion_matrix(actualTopicLabels,predictedTopicLabels , labels=aspects))
# Printing the precision and recall, among other metrics
print(metrics.classification_report(actualTopicLabels,predictedTopicLabels, labels=aspects))

[[83  6  6]
 [39 76  8]
 [29  0 71]]
              precision    recall  f1-score   support

     battery       0.55      0.87      0.67        95
      screen       0.93      0.62      0.74       123
      camera       0.84      0.71      0.77       100

    accuracy                           0.72       318
   macro avg       0.77      0.73      0.73       318
weighted avg       0.79      0.72      0.73       318



In [323]:
from imblearn.metrics import sensitivity_specificity_support
y_true = np.array(actualTopicLabels)
y_pred = np.array(predictedTopicLabels)
sensitivity_specificity_support(y_true, y_pred, average='macro')

(0.7338567964627014, 0.8666926167728407, None)

In [6]:
keywords=['port','drain','battery','charge','charger','charging point','time','screen','display','button','glass','touch','keyboard','camera','quality','size','picture','pictures','photos']
lkeys=len(keywords)

In [7]:
matrix=np.zeros((lkeys+sentences_count,lkeys+sentences_count))

In [8]:
pprint(unique_aspSent)

['Another thing, it says it comes with a charger and it didnt!',
 '!battery  is average',
 'Iphone came in very good condition as described, but the charger was not '
 'working well.',
 'Besides that, everything was ok.battery is worst.battery is draining very '
 'fast',
 'The microphone wouldnt work half the time and the person couldnt hear me.',
 'Got this phone because my android smart phone would only hold a charge for 5 '
 'hours.',
 'On the plus side, this phone holds a charge for days, on the negative side, '
 'folks dont get my texts sometimes, or I dont get their texts sometimes, and '
 'it ALWAYS cuts my call off when Ive only been talking for a few minutes.',
 'This one is garbage, sorry I bought it.battery is draining very fast and '
 'worst battery.',
 'The purchase of this iphone 4 has caused me so much hassle and waste of '
 'time.',
 'I had contacted mobile karma and they have no means to settling or solving '
 'this issue.it is taking lot of time for getting charged an

In [9]:
l=sentences_count+lkeys
row_names=list(range(l))
column_names=list(range(l))
adjMatrix=pd.DataFrame(matrix,index=row_names,columns=column_names)
adjMatrix

,0,1,2,3,4,5,6,7,8,9,...,536,537,538,539,540,541,542,543,544,545
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
542,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
543,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
544,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# absa

for i in range(sentences_count+lkeys):
    if i< lkeys:
        continue
    sentence=unique_aspSent[i-lkeys]
    j=0
    for key in keywords:
        aspect=""
        for k,v in keys.items():
            if key in keys[k]:
                aspect=k
                if sentence in absa[aspect]:
                    keywords.index(aspect)
                    adjMatrix[i][j]=1
        j+=1
        
pprint(adjMatrix)

     0    1    2    3    4    5    6    7    8    9    ...  536  537  538  \
0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
1    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
2    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
3    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
4    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
..   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
541  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
542  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
543  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
544  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
545  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

     539  540  541  542  543  544  545  
0    0.0  0.0  0.0  1.0  0.0  0.0 

In [11]:
from igraph import Graph

In [12]:
graph=Graph.Adjacency(adjMatrix.values.tolist())

In [13]:
weights=graph.pagerank(vertices=list(range(lkeys)), directed=False, damping=0.85,weights=None, arpack_options=None, implementation="prpack",niter=1000, eps=0.001)
weights

[0.030018680851324964,
 0.030018680851324787,
 0.03001868085132461,
 0.03001868085132442,
 0.030018680851324215,
 0.030018680851324007,
 0.030018680851323788,
 0.022275217773405024,
 0.022275217773404823,
 0.022275217773404615,
 0.022275217773404396,
 0.022275217773404177,
 0.022275217773403955,
 0.01974981494687926,
 0.01974981494687906,
 0.019749814946878844,
 0.019749814946878632,
 0.019749814946878413,
 0.019749814946878188]

In [312]:
predictedTopicLabels=[]
value=dict();
weight={"battery":0.029,"charge":0.029,"charging port": 0.028,"drain":0.029,"port": 0.029,"charger":0.028,"camera":0.0198,"picture":0.01968,"pictures":0.01968,"quality":0.01968,"photos":0.0198,"screen":0.024,"time":0.029,"touch":0.024,"button":0.0242,"display":0.0242,"glass":0.025,"keyboard":0.024};
keywords=['port','drain','battery','charge','charger','charging point','time','screen','display','button','glass','touch','keyboard','camera','quality','size','picture','pictures','photos']

topic=""
for review in data:
    value['battery']=0;value['screen']=0;value['camera']=0;
    maxi=0
    for k,v in keys.items():
        cnt = list(set(v).intersection(set(review.split())))
        for key in cnt:
            value[k]+=weight[key]      
    predictedTopicLabels.append(max(value,key = value.get))
   
        
len(predictedTopicLabels)    

318

In [314]:
c=0
for i in range(318):
    print(actualTopicLabels[i].lower(),predictedTopicLabels[i],actualTopicLabels[i].lower()==predictedTopicLabels[i])
    if actualTopicLabels[i].lower()==predictedTopicLabels[i]:
        c+=1
print(c)

battery battery True
battery battery True
camera camera True
screen battery False
screen battery False
camera camera True
screen camera False
camera camera True
camera camera True
screen battery False
screen battery False
camera battery False
battery battery True
battery battery True
camera camera True
battery battery True
screen screen True
camera battery False
screen screen True
screen screen True
screen camera False
camera camera True
battery battery True
battery battery True
camera battery False
battery screen False
battery camera False
camera battery False
screen screen True
camera camera True
battery battery True
battery camera False
battery battery True
screen battery False
screen camera False
screen battery False
screen screen True
battery battery True
camera screen False
battery battery True
camera camera True
camera camera True
camera battery False
battery battery True
camera battery False
battery battery True
screen screen True
camera camera True
camera camera True
screen sc

In [315]:
#confusion matrix
from sklearn import metrics
print(metrics.confusion_matrix(actualTopicLabels,predictedTopicLabels , labels=aspects))
# Printing the precision and recall, among other metrics
print(metrics.classification_report(actualTopicLabels,predictedTopicLabels, labels=aspects))

[[83  6  6]
 [39 74 10]
 [35  3 62]]
              precision    recall  f1-score   support

     battery       0.53      0.87      0.66        95
      screen       0.89      0.60      0.72       123
      camera       0.79      0.62      0.70       100

    accuracy                           0.69       318
   macro avg       0.74      0.70      0.69       318
weighted avg       0.75      0.69      0.69       318



In [319]:
from imblearn.metrics import sensitivity_specificity_support
y_true = np.array(actualTopicLabels)
y_pred = np.array(predictedTopicLabels)
sensitivity_specificity_support(y_true, y_pred, average='macro')

(0.6984367422621595, 0.8495376978036294, None)

In [260]:
from imblearn.metrics import sensitivity_specificity_support


In [316]:
########Sentimental Inference################

predictedSentimentLabels=[]
predictedRatings=[]
for review in testData.Reviews.values.tolist():
    topicSentences=[]
    s=sent_tokenize(review)
    for sentence in s:
        if any(word in sentence for word in keywords):
            topicSentences.append(sentence)
    temp=0
    s1=''.join(topicSentences)
    blob=TextBlob(s1)
    for sent in blob.sentences:
        score = sent.sentiment.polarity
        temp+=score
        
    l=len(topicSentences)
    if l>0:
        temp/=l
    
    if temp==0:
        predictedRatings.append(3)
        predictedSentimentLabels.append("Neutral")
    elif temp>0:
        if temp>0.5:
            predictedRatings.append(5)
        else:
            predictedRatings.append(4)
        predictedSentimentLabels.append("Positive")
    else:
        if temp<-0.5:
            predictedRatings.append(1)
        else:
            predictedRatings.append(2)
        predictedSentimentLabels.append("Negative")

print(predictedSentimentLabels,predictedRatings) 


['Negative', 'Positive', 'Positive', 'Positive', 'Negative', 'Negative', 'Negative', 'Positive', 'Negative', 'Neutral', 'Positive', 'Positive', 'Negative', 'Positive', 'Positive', 'Negative', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Negative', 'Positive', 'Positive', 'Neutral', 'Positive', 'Positive', 'Negative', 'Positive', 'Neutral', 'Negative', 'Positive', 'Positive', 'Negative', 'Neutral', 'Negative', 'Neutral', 'Positive', 'Negative', 'Positive', 'Positive', 'Positive', 'Negative', 'Positive', 'Positive', 'Negative', 'Positive', 'Positive', 'Positive', 'Negative', 'Positive', 'Neutral', 'Positive', 'Positive', 'Positive', 'Negative', 'Positive', 'Negative', 'Positive', 'Positive', 'Neutral', 'Negative', 'Negative', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Negative', 'Negative', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Negative', 'Positive', 'Negative', 'Positive', 'Positive', 'Positive', 'Positive', 'Neutral',

In [317]:
c=0
for i in range(318):
    print(actualSentimentLabels[i],predictedSentimentLabels[i],actualSentimentLabels[i]==predictedSentimentLabels[i])
    if actualSentimentLabels[i]==predictedSentimentLabels[i]:
        c+=1
print(c)

Neutral Negative False
Negative Positive False
Negative Positive False
Positive Positive True
Negative Negative True
Negative Negative True
Neutral Negative False
Positive Positive True
Negative Negative True
Positive Neutral False
Positive Positive True
Positive Positive True
Negative Negative True
Negative Positive False
Positive Positive True
Negative Negative True
Positive Positive True
Positive Positive True
Neutral Positive False
Neutral Positive False
Positive Positive True
Positive Positive True
Negative Negative True
Positive Positive True
Positive Positive True
Positive Neutral False
Positive Positive True
Positive Positive True
Negative Negative True
Positive Positive True
Positive Neutral False
Negative Negative True
Neutral Positive False
Positive Positive True
Negative Negative True
Positive Neutral False
Neutral Negative False
Negative Neutral False
Neutral Positive False
Negative Negative True
Neutral Positive False
Positive Positive True
Positive Positive True
Negative

In [318]:
from sklearn import metrics
print(metrics.confusion_matrix(actualSentimentLabels,predictedSentimentLabels , labels=['Positive','Negative','Neutral']))
# Printing the precision and recall, among other metrics
print(metrics.classification_report(actualSentimentLabels,predictedSentimentLabels, labels=['Positive','Negative','Neutral']))

[[129   4  17]
 [ 36  62  16]
 [ 29  19   6]]
              precision    recall  f1-score   support

    Positive       0.66      0.86      0.75       150
    Negative       0.73      0.54      0.62       114
     Neutral       0.15      0.11      0.13        54

    accuracy                           0.62       318
   macro avg       0.52      0.50      0.50       318
weighted avg       0.60      0.62      0.60       318



In [325]:

from imblearn.metrics import sensitivity_specificity_support
y_true = np.array(actualSentimentLabels)
y_pred = np.array(predictedSentimentLabels)
sensitivity_specificity_support(y_true, y_pred, average='macro')

(0.504990253411306, 0.7917833800186741, None)

In [15]:
print(overall_rating)

    
    

3.9
